In [ ]:
# Dependencies
import pandas as pd
import numpy as np
import json
import matplotlib
# from sodapy import Socrata
from config import census_key
import pprint
import requests

In [ ]:
def json_to_dataframe(response):
    return pd.DataFrame(response.json()[1:], columns=response.json()[0])

In [ ]:
cdc_data = pd.read_csv("../Project_1/cdc_data.csv")

cdc_df = pd.DataFrame(cdc_data)
cdc_df.columns
cdc_df = cdc_df[['case_month', 'res_state', 'res_county', 'age_group', 'sex', 'race', 'ethnicity', 'current_status']]
# cdc_df.value_counts('race')
cdc_df.head(100)



In [ ]:
# Census data 2019
url = "https://api.census.gov/data/2019/acs/acs5?get=NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,B23025_005E&for=county:*&in=state:27&key={0}".format(census_key)
response = requests.request("GET", url)

census_df_2019 = json_to_dataframe(response)

# Split NAME into county & state
name = census_df_2019['NAME'].str.split(",", n=1, expand=True)
COUNTY = census_df_2019['County']=name[0]
STATE = census_df_2019['State']=name[1]
census_df_2019.drop(columns=["NAME"], inplace=True)

# Rename columns to something intelligible
census_df_2019 = census_df_2019[['County', 'State', 'B19013_001E', 'B01003_001E', 'B01002_001E', 'B19301_001E', 'B17001_002E', 'B23025_005E']]
census_df_2019 = census_df_2019.rename(columns={
    'B01003_001E':'Population',
    'B01002_001E': 'Median Age',
    'B19013_001E': 'Household Income',
    'B19301_001E': 'Per Capita Income',
    'B17001_002E': 'Poverty Count',
    'B23025_005E': 'Unemployment Count'
})
# Remove ...County from County in dataframe
census_df_2019['County'] = census_df_2019['County'].replace('County', '', regex=True)

# Upper case for County
census_df_2019['County'] = census_df_2019['County'].str.upper()


census_df_2019['Year'] = 2019
census_df_2019

In [ ]:
# Census data 2020
url = "https://api.census.gov/data/2020/acs/acs5?get=NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,B23025_005E&for=county:*&in=state:27&key={0}".format(census_key)
response = requests.request("GET", url)

census_df_2020 = json_to_dataframe(response)

# Split NAME into county & state
name = census_df_2020['NAME'].str.split(",", n=1, expand=True)
COUNTY = census_df_2020['County']=name[0]
STATE = census_df_2020['State']=name[1]
census_df_2020.drop(columns=["NAME"], inplace=True)

# Rename columns to something intelligible
census_df_2020 = census_df_2020[['County', 'State', 'B19013_001E', 'B01003_001E', 'B01002_001E', 'B19301_001E', 'B17001_002E', 'B23025_005E']]
census_df_2020 = census_df_2020.rename(columns={
    'B01003_001E':'Population',
    'B01002_001E': 'Median Age',
    'B19013_001E': 'Household Income',
    'B19301_001E': 'Per Capita Income',
    'B17001_002E': 'Poverty Count',
    'B23025_005E': 'Unemployment Count'
})

# Remove ...County from County in dataframe
census_df_2020['County'] = census_df_2020['County'].replace('County', '', regex=True)

# Upper case for County
census_df_2020['County'] = census_df_2020['County'].str.upper()

census_df_2020['Year'] = 2020
census_df_2020
